In [335]:
import spacy
import nltk
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, VectorizerMixin
from utils import *

In [334]:
%reload_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_colwidth', 120)

In [8]:
# Note: you can add other languages that Spacy supports, or download
# larger models for english that Spacy offers. 
nlp = spacy.load('en') 

In [450]:
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

def stop_word_removal(li):    
    return [l for l in li if l not in ENGLISH_STOP_WORDS]

In [451]:
from utils import clean_html
from sklearn.feature_extraction.text import strip_accents_unicode


def clean_twitter(s):
    """ Cleans Twitter specific issues 
    
    Can you think of what else you might need to add here?
    """
    s = sub(r'@\w+', '', s) #remove @ mentions from tweets    
    return s

def preprocessor(s):
    """ For all basic string cleanup. 
    
    Think of what you can add to this to improve things. What is
    specific to your goal, how can you transform the text. Add tokens,
    remove things, unify things. 
    """
    s = clean_html(s)
    s = strip_accents_unicode(s.lower())
    s = clean_twitter(s)
    return s

In [452]:
import spacy

def cool_tokenizer(sent):
    """ Idea from Travis in class: adds a token to the end with nsubj and root verb!"""
    doc = nlp(sent)
    tokens = sorted(doc, key = lambda t: t.dep_)
    return ' '.join([t.lemma_ for t in tokens if t.dep_ in ['nsubj', 'ROOT']])

cool_tokenizer('a migrant died in crossing the river')

'die migrant'

In [453]:
from langdetect.lang_detect_exception import LangDetectException
from langdetect import detect

def dep_tokenizer(sent):
    """ A simple version of tokenzing with the dependencies.
    
    Note: this is monolingual! Also, it still doesn't take into 
    account correlations!
    """
    doc = nlp(sent)
    tokens = [t for t in doc if not t.is_stop and t.dep_ not in ['punct', '']]
    return [':'.join([t.lemma_,t.dep_]) for t in tokens]

dep_tokenizer('a migrant died in crossing the river')

['migrant:nsubj', 'die:ROOT', 'cross:pcomp', 'river:dobj']

In [454]:
import re
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

def analyzer(s, ngram_range = (1,2)):
    """ Does everything to turn raw documents into tokens.  
    
    Note: None of the above tokenizers are implemented!
    """
    s = preprocessor(s)
    pattern = re.compile(r"(?u)\b\w\w+\b")
    unigrams = pattern.findall(s)
    unigrams = [u for u in unigrams if u not in ENGLISH_STOP_WORDS]
    unigrams = dep_tokenizer(s)
    tokens = ngrammer(unigrams, ngram_range)
    return tokens

In [455]:
X = pd.read_csv('kaggle/train.csv').tweet
y = pd.read_csv('kaggle/train.csv').label

cutoff = 1750
X_train, X_test, y_train, y_test = X[0:cutoff], X[cutoff:], y[0:cutoff], y[cutoff:]

In [456]:
X_test.shape, y_test.shape

((450,), (450,))

In [457]:
def create_vectors(X_train, X_test, analyzer = analyzer):
    """ Just a small helper function that applies the SKLearn Vectorizer with our analyzer """
    idx = X_train.shape[0]
    X = pd.concat([X_train, X_test])
    vectorizer = TfidfVectorizer(analyzer=analyzer).fit(X)
    vector = vectorizer.transform(X)
    return vector[0:idx], vector[idx:], vectorizer

In [461]:
V_train, V_test, vectorizer = create_vectors(X_train, X_test)

In [462]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import recall_score, precision_score, f1_score, average_precision_score, roc_auc_score

model = MultinomialNB(class_prior=[0.5,0.5])
model.fit(V_train, y_train)
preds = model.predict_proba(V_test)[:,1]
roc_auc_score(y_test, preds)

0.55158023309074256

In [463]:
from sklearn.svm import LinearSVC

model = LinearSVC(tol = 10e-6, max_iter = 8000)
model.fit(V_train, y_train)
preds = model.decision_function(V_test)
# scorer(y_test, preds)
roc_auc_score(y_test, preds)

0.69148557178482362

In [273]:
# Look at your false predictions!
false_pos, false_neg = get_errors(X_test, y_test, preds)

## Submission!

Here you can make the submission required for Kaggle. 

In [459]:
test_df = pd.read_csv('kaggle/test.csv')
X_sub, id_sub = test_df.tweet, test_df.id
V_train, V_test, _ = create_vectors(X, X_sub)
model.fit(V_train, y)
preds = model.decision_function(V_test)

array([-0.62006737, -0.39139947, -0.51430024, -0.1642042 , -0.50812492,
       -0.69595094, -0.59708002, -0.46660077, -0.49220586, -0.45437326,
       -0.62704535,  0.03460983, -0.55639225, -0.32248204, -0.62315828,
       -0.68500952, -0.57469783,  0.18942113, -0.5753778 , -0.53782775,
       -0.60610609, -0.51022849, -0.581047  , -0.52080148, -0.66895792,
       -0.381175  , -0.50118118, -0.75417221, -0.5022477 , -0.62979106,
       -0.53121564, -0.54735739, -0.36690684, -0.37127499, -0.57469783,
       -0.77127448, -0.59188201, -0.60184589, -0.55504831, -0.55836117,
       -0.51430024, -0.51963622, -0.56631437, -0.6095309 , -0.57168439,
       -0.46441326, -0.66964507, -0.41775753, -0.60939078, -0.07629341,
       -0.60063491, -0.46702252, -0.57138097, -0.31722487, -0.57176085,
       -0.37631394, -0.53054909, -0.38780435, -0.69517749, -0.74392093,
       -0.60577324, -0.59627703, -0.58242079, -0.55624438, -0.62286286,
       -0.55394184, -0.4927646 , -0.47757522, -0.62903026, -0.60

In [460]:
write_submission_csv(preds, id_sub, 'kaggle/submission.csv')